In [2]:
from google.colab import drive
drive.mount("/content/drive")


import pandas as pd
import numpy as np
from pathlib import Path

INTERIM = Path("/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/interim")

cbf = pd.read_parquet(INTERIM / "cbf_2003_2023.parquet").copy()
fd  = pd.read_parquet(INTERIM / "fd_2012_2023.parquet").copy()

# --- recorte em comum ---
cbf = cbf[cbf["ano_campeonato"].between(2012, 2023)].copy()

print("CBF (2012–2023):", cbf.shape)
print("FD  (2012–2023):", fd.shape)

# --- função de normalização de texto (para times) ---
import re
def norm_team(s: str) -> str:
    if pd.isna(s):
        return s
    s = str(s).strip().lower()
    s = re.sub(r"\s+", " ", s)             # espaços
    s = s.replace("á","a").replace("ã","a").replace("â","a")
    s = s.replace("é","e").replace("ê","e")
    s = s.replace("í","i")
    s = s.replace("ó","o").replace("ô","o")
    s = s.replace("ú","u")
    s = s.replace("ç","c")
    s = s.replace("-", " ")
    return s

for df in (cbf, fd):
    df["home_norm"] = df["time_mandante"].map(norm_team)
    df["away_norm"] = df["time_visitante"].map(norm_team)

# --- 1) cobertura por ano (jogos) ---
cov = pd.DataFrame({
    "cbf_games": cbf.groupby("ano_campeonato").size(),
    "fd_games":  fd.groupby("ano_campeonato").size()
}).fillna(0).astype(int)

print("\n--- COBERTURA POR ANO (2012–2023) ---")
display(cov)

# --- 2) comparação de conjuntos de times ---
cbf_teams = set(pd.concat([cbf["home_norm"], cbf["away_norm"]]).unique())
fd_teams  = set(pd.concat([fd["home_norm"],  fd["away_norm"]]).unique())

only_cbf = sorted(cbf_teams - fd_teams)
only_fd  = sorted(fd_teams - cbf_teams)

print("\nTimes somente na CBF (normalizados):", len(only_cbf))
print(only_cbf[:50])
print("\nTimes somente no FD (normalizados):", len(only_fd))
print(only_fd[:50])

# --- 3) match de partidas por (data + mandante + visitante) ---
# Normalizar data: garantir apenas date (sem hora)
cbf["date_only"] = pd.to_datetime(cbf["data"]).dt.date
fd["date_only"]  = pd.to_datetime(fd["data"]).dt.date

m = cbf.merge(
    fd,
    on=["date_only", "home_norm", "away_norm"],
    suffixes=("_cbf", "_fd"),
    how="inner"
)

print("\n--- MATCH por data+mandante+visitante ---")
print("Matches encontrados:", m.shape[0])
print("CBF total (2012–2023):", cbf.shape[0])
print("FD total  (2012–2023):", fd.shape[0])
print("Cobertura CBF via match:", round(m.shape[0] / cbf.shape[0], 4))
print("Cobertura FD  via match:", round(m.shape[0] / fd.shape[0], 4))

# --- 4) divergência de placar nos matches ---
m["score_equal"] = (m["gols_mandante_cbf"] == m["gols_mandante_fd"]) & (m["gols_visitante_cbf"] == m["gols_visitante_fd"])
diff_scores = m[~m["score_equal"]].copy()

print("\n--- CONSISTÊNCIA DE PLACAR (matches) ---")
print("Placares iguais:", int(m["score_equal"].sum()))
print("Placares diferentes:", diff_scores.shape[0])

if diff_scores.shape[0] > 0:
    display(diff_scores[[
        "ano_campeonato_cbf","data_cbf","time_mandante_cbf","time_visitante_cbf",
        "gols_mandante_cbf","gols_visitante_cbf",
        "gols_mandante_fd","gols_visitante_fd"
    ]].head(20))

# --- 5) onde estão os não-matcheados? (diagnóstico) ---
cbf_key = cbf[["date_only","home_norm","away_norm"]].drop_duplicates()
fd_key  = fd[["date_only","home_norm","away_norm"]].drop_duplicates()

cbf_unmatched = cbf_key.merge(fd_key, on=["date_only","home_norm","away_norm"], how="left", indicator=True)
cbf_unmatched = cbf_unmatched[cbf_unmatched["_merge"] == "left_only"].drop(columns=["_merge"])

fd_unmatched = fd_key.merge(cbf_key, on=["date_only","home_norm","away_norm"], how="left", indicator=True)
fd_unmatched = fd_unmatched[fd_unmatched["_merge"] == "left_only"].drop(columns=["_merge"])

print("\n--- NÃO MATCHEADOS ---")
print("CBF não matcheados:", cbf_unmatched.shape[0])
print("FD  não matcheados:", fd_unmatched.shape[0])

# amostras para você ver padrão de divergência
display(cbf_unmatched.head(10))
display(fd_unmatched.head(10))


Mounted at /content/drive
CBF (2012–2023): (4559, 7)
FD  (2012–2023): (4559, 7)

--- COBERTURA POR ANO (2012–2023) ---


,cbf_games,fd_games
ano_campeonato,,
2012,380,380
2013,380,380
2014,380,380
2015,380,380
2016,379,379
2017,380,380
2018,380,380
2019,380,380
2020,380,380



Times somente na CBF (normalizados): 17
['atletico pr', 'avai fc', 'botafogo', 'ceara sc', 'chapecoense', 'coritiba fc', 'criciuma ec', 'cuiaba mt', 'ec bahia', 'ec vitoria', 'figueirense fc', 'flamengo', 'goias ec', 'joinville sc', 'rb bragantino', 'santos fc', 'vasco da gama']

Times somente no FD (normalizados): 14
['avai', 'bahia', 'botafogo rj', 'bragantino', 'ceara', 'chapecoense sc', 'coritiba', 'criciuma', 'cuiaba', 'figueirense', 'flamengo rj', 'joinville', 'vasco', 'vitoria']

--- MATCH por data+mandante+visitante ---
Matches encontrados: 1172
CBF total (2012–2023): 4559
FD total  (2012–2023): 4559
Cobertura CBF via match: 0.2571
Cobertura FD  via match: 0.2571

--- CONSISTÊNCIA DE PLACAR (matches) ---
Placares iguais: 1172
Placares diferentes: 0

--- NÃO MATCHEADOS ---
CBF não matcheados: 3387
FD  não matcheados: 3387


,date_only,home_norm,away_norm
0,2012-05-19,figueirense fc,nautico
2,2012-05-19,sport recife,flamengo
3,2012-05-20,botafogo,sao paulo
5,2012-05-20,internacional,coritiba fc
7,2012-05-20,vasco da gama,gremio
8,2012-05-20,ec bahia,santos fc
11,2012-05-26,nautico,cruzeiro
12,2012-05-26,flamengo,internacional
13,2012-05-26,portuguesa,vasco da gama
14,2012-05-27,fluminense,figueirense fc


,date_only,home_norm,away_norm
1,2012-05-19,sport recife,flamengo rj
2,2012-05-20,figueirense,nautico
3,2012-05-20,botafogo rj,sao paulo
5,2012-05-20,internacional,coritiba
7,2012-05-20,bahia,santos
9,2012-05-20,vasco,gremio
11,2012-05-26,flamengo rj,internacional
12,2012-05-26,portuguesa,vasco
13,2012-05-27,nautico,cruzeiro
15,2012-05-27,coritiba,botafogo rj


In [3]:
# =========================
# Canonicalização de clubes (por região)
# Objetivo: alinhar nomes entre CBF x football-data
# =========================

TEAM_ALIASES = {
    # -------------------------
    # SUDESTE — RJ
    # -------------------------
    "flamengo rj": "flamengo",
    "botafogo rj": "botafogo",

    # -------------------------
    # SUDESTE — SP
    # -------------------------
    "santos fc": "santos",
    # (sao paulo / palmeiras / corinthians já costumam vir iguais, mas manter não faz mal)
    "sao paulo": "sao paulo",
    "palmeiras": "palmeiras",
    "corinthians": "corinthians",

    # -------------------------
    # SUDESTE — MG
    # -------------------------
    "atletico mg": "atletico mineiro",

    # -------------------------
    # SUL — PR / SC
    # -------------------------
    # PR
    "atletico pr": "athletico paranaense",
    "athletico pr": "athletico paranaense",
    "coritiba fc": "coritiba",
    # SC
    "avai fc": "avai",
    "figueirense fc": "figueirense",
    "criciuma ec": "criciuma",
    "chapecoense sc": "chapecoense",

    # -------------------------
    # SUL — RS
    # -------------------------
    "gremio": "gremio",
    "internacional": "internacional",

    # -------------------------
    # CENTRO-OESTE
    # -------------------------
    "cuiaba mt": "cuiaba",

    # -------------------------
    # NORDESTE
    # -------------------------
    "ec bahia": "bahia",
    "ec vitoria": "vitoria",
    "ceara sc": "ceara",
    "sport recife": "sport",
    "nautico": "nautico",

    # -------------------------
    # AJUSTES “NOMES CURTOS” (FD → canônico)
    # (Deixa explícito para auditoria)
    # -------------------------
    "avai": "avai",
    "bahia": "bahia",
    "bragantino": "bragantino",
    "ceara": "ceara",
    "chapecoense sc": "chapecoense",
    "coritiba": "coritiba",
    "criciuma": "criciuma",
    "cuiaba": "cuiaba",
    "figueirense": "figueirense",
    "flamengo": "flamengo",
    "joinville": "joinville",
    "vasco": "vasco",
    "vitoria": "vitoria",

    # -------------------------
    # AJUSTES “NOMES LONGOS” (CBF → canônico)
    # -------------------------
    "rb bragantino": "bragantino",
    "vasco da gama": "vasco",
    "joinville sc": "joinville",
    "botafogo": "botafogo",
    "santos": "santos",
}


In [4]:
def canon_team(name: str) -> str:
    import pandas as pd
    if pd.isna(name):
        return name
    n = str(name).lower().strip()
    n = n.replace("-", " ")
    n = n.replace("á","a").replace("ã","a").replace("â","a")
    n = n.replace("é","e").replace("ê","e")
    n = n.replace("í","i")
    n = n.replace("ó","o").replace("ô","o")
    n = n.replace("ú","u")
    n = n.replace("ç","c")
    n = " ".join(n.split())
    return TEAM_ALIASES.get(n, n)


In [5]:
for df in (cbf, fd):
    df["home_norm"] = df["time_mandante"].map(canon_team)
    df["away_norm"] = df["time_visitante"].map(canon_team)


In [6]:
print("home_norm existe?", "home_norm" in cbf.columns, " | ", "home_norm" in fd.columns)
print("away_norm existe?", "away_norm" in cbf.columns, " | ", "away_norm" in fd.columns)

print("\nNulos após canon (CBF):",
      cbf["home_norm"].isna().sum(), cbf["away_norm"].isna().sum())
print("Nulos após canon (FD):",
      fd["home_norm"].isna().sum(), fd["away_norm"].isna().sum())


home_norm existe? True  |  True
away_norm existe? True  |  True

Nulos após canon (CBF): 0 0
Nulos após canon (FD): 0 0


In [7]:
print("\nCBF - amostra antes/depois")
display(cbf[["time_mandante","home_norm","time_visitante","away_norm"]].sample(10, random_state=42))

print("\nFD - amostra antes/depois")
display(fd[["time_mandante","home_norm","time_visitante","away_norm"]].sample(10, random_state=42))



CBF - amostra antes/depois


,time_mandante,home_norm,time_visitante,away_norm
4081,Atlético-MG,atletico mineiro,Atlético-PR,athletico paranaense
7087,Sport Recife,sport,Cuiabá-MT,cuiaba
3633,Vasco da Gama,vasco,Náutico,nautico
7198,Grêmio,gremio,Corinthians,corinthians
3745,São Paulo,sao paulo,Sport Recife,sport
6002,Internacional,internacional,Vasco da Gama,vasco
7581,Goiás,goias,Fluminense,fluminense
5990,Atlético-MG,atletico mineiro,Fluminense,fluminense
5012,Ponte Preta,ponte preta,Fluminense,fluminense
7775,Palmeiras,palmeiras,América-MG,america mg



FD - amostra antes/depois


,time_mandante,home_norm,time_visitante,away_norm
471,Atletico-MG,atletico mineiro,Athletico-PR,athletico paranaense
3477,Athletico-PR,athletico paranaense,Chapecoense-SC,chapecoense
23,Coritiba,coritiba,Portuguesa,portuguesa
3588,Gremio,gremio,Corinthians,corinthians
135,Sao Paulo,sao paulo,Sport Recife,sport
2392,Sport Recife,sport,Gremio,gremio
3971,Goias,goias,Fluminense,fluminense
2380,Vasco,vasco,Sport Recife,sport
1402,Ponte Preta,ponte preta,Fluminense,fluminense
4165,Juventude,juventude,Flamengo RJ,flamengo


In [8]:
cbf_teams = set(pd.concat([cbf["home_norm"], cbf["away_norm"]]).unique())
fd_teams  = set(pd.concat([fd["home_norm"],  fd["away_norm"]]).unique())

only_cbf = sorted(cbf_teams - fd_teams)
only_fd  = sorted(fd_teams - cbf_teams)

print("Times somente na CBF (após canon):", len(only_cbf))
print(only_cbf[:50])

print("\nTimes somente no FD (após canon):", len(only_fd))
print(only_fd[:50])


Times somente na CBF (após canon): 1
['goias ec']

Times somente no FD (após canon): 0
[]


In [9]:
cbf["date_only"] = pd.to_datetime(cbf["data"]).dt.date
fd["date_only"]  = pd.to_datetime(fd["data"]).dt.date

print("CBF date_only:", cbf["date_only"].min(), "→", cbf["date_only"].max())
print("FD  date_only:", fd["date_only"].min(), "→", fd["date_only"].max())


CBF date_only: 2012-05-19 → 2023-12-07
FD  date_only: 2012-05-19 → 2023-12-07


In [10]:
cbf_key = cbf[["date_only","home_norm","away_norm"]].drop_duplicates()
fd_key  = fd[["date_only","home_norm","away_norm"]].drop_duplicates()

keys_intersection = cbf_key.merge(fd_key, on=["date_only","home_norm","away_norm"], how="inner")

print("Chaves CBF:", cbf_key.shape[0])
print("Chaves FD :", fd_key.shape[0])
print("Interseção:", keys_intersection.shape[0])
print("Cobertura prévia:", round(keys_intersection.shape[0] / cbf_key.shape[0], 4))


Chaves CBF: 4559
Chaves FD : 4559
Interseção: 3487
Cobertura prévia: 0.7649


In [11]:
TEAM_ALIASES["goias ec"] = "goias"


In [12]:
for df in (cbf, fd):
    df["home_norm"] = df["time_mandante"].map(canon_team)
    df["away_norm"] = df["time_visitante"].map(canon_team)


In [13]:
import pandas as pd

# chaves
cbf_key = cbf[["date_only","home_norm","away_norm"]].drop_duplicates().copy()
fd_key  = fd[["date_only","home_norm","away_norm"]].drop_duplicates().copy()

# gerar versões deslocadas da data do FD (±1)
fd_key_m1 = fd_key.copy()
fd_key_m1["date_only"] = pd.to_datetime(fd_key_m1["date_only"]) - pd.Timedelta(days=1)
fd_key_m1["date_only"] = fd_key_m1["date_only"].dt.date

fd_key_p1 = fd_key.copy()
fd_key_p1["date_only"] = pd.to_datetime(fd_key_p1["date_only"]) + pd.Timedelta(days=1)
fd_key_p1["date_only"] = fd_key_p1["date_only"].dt.date

# interseções
exact = cbf_key.merge(fd_key, on=["date_only","home_norm","away_norm"], how="inner")
minus1 = cbf_key.merge(fd_key_m1, on=["date_only","home_norm","away_norm"], how="inner")
plus1  = cbf_key.merge(fd_key_p1, on=["date_only","home_norm","away_norm"], how="inner")

print("Match exato:", exact.shape[0], "| cobertura:", round(exact.shape[0]/cbf_key.shape[0], 4))
print("Match com FD -1 dia:", minus1.shape[0], "| cobertura:", round(minus1.shape[0]/cbf_key.shape[0], 4))
print("Match com FD +1 dia:", plus1.shape[0], "| cobertura:", round(plus1.shape[0]/cbf_key.shape[0], 4))

# união (evitar contar duplicado)
all_matches = pd.concat([exact, minus1, plus1]).drop_duplicates()
print("\nMatch total (exato ou ±1 dia):", all_matches.shape[0], "| cobertura:", round(all_matches.shape[0]/cbf_key.shape[0], 4))


Match exato: 3644 | cobertura: 0.7993
Match com FD -1 dia: 127 | cobertura: 0.0279
Match com FD +1 dia: 780 | cobertura: 0.1711

Match total (exato ou ±1 dia): 4551 | cobertura: 0.9982


In [14]:
# não matcheados (match exato) — após corrigir Goiás e reaplicar
cbf_unmatched = cbf_key.merge(fd_key, on=["date_only","home_norm","away_norm"], how="left", indicator=True)
cbf_unmatched = cbf_unmatched[cbf_unmatched["_merge"]=="left_only"].drop(columns=["_merge"])

print("CBF não matcheados (exato):", cbf_unmatched.shape[0])
display(cbf_unmatched.sample(20, random_state=42))


CBF não matcheados (exato): 915


,date_only,home_norm,away_norm
2146,2017-09-25,atletico mineiro,vitoria
4092,2022-10-06,bragantino,cuiaba
2069,2017-08-03,chapecoense,bahia
2075,2017-08-04,sao paulo,coritiba
2087,2017-08-07,avai,santos
2575,2018-10-22,cruzeiro,chapecoense
3236,2020-11-14,coritiba,bahia
2896,2019-10-11,avai,vasco
2766,2019-07-22,avai,goias
118,2012-07-26,flamengo,portuguesa


In [15]:
# pegar uma amostra e tentar localizar no FD por time (data próxima)
sample = cbf_unmatched.sample(15, random_state=7)

fd_lookup = fd_key.copy()
fd_lookup["date_dt"] = pd.to_datetime(fd_lookup["date_only"])

out = []
for _, r in sample.iterrows():
    d = pd.to_datetime(r["date_only"])
    cand = fd_lookup[
        (fd_lookup["home_norm"] == r["home_norm"]) &
        (fd_lookup["away_norm"] == r["away_norm"]) &
        (fd_lookup["date_dt"].between(d - pd.Timedelta(days=2), d + pd.Timedelta(days=2)))
    ]
    out.append({
        "cbf_date": r["date_only"],
        "home": r["home_norm"],
        "away": r["away_norm"],
        "fd_candidates_dates": cand["date_only"].tolist()
    })

pd.DataFrame(out)


,cbf_date,home,away,fd_candidates_dates
0,2012-09-05,cruzeiro,botafogo,[2012-09-06]
1,2020-11-14,coritiba,bahia,[2020-11-16]
2,2013-11-14,sao paulo,flamengo,[2013-11-13]
3,2012-08-29,nautico,figueirense,[2012-08-30]
4,2022-07-21,athletico paranaense,atletico go,[2022-07-20]
5,2022-11-10,fortaleza,bragantino,[2022-11-09]
6,2021-02-22,goias,bragantino,[2021-02-21]
7,2022-07-31,corinthians,botafogo,[2022-07-30]
8,2013-05-30,botafogo,santos,[2013-05-29]
9,2017-11-09,athletico paranaense,corinthians,[2017-11-08]
